In [1]:
import tensorflow as tf
import tensorflow.keras.layers as tfl
import os
import numpy as np
import datetime
import scipy.io
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [2]:
url = "https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/wiki_crop.tar"

dataset = tf.keras.utils.get_file(
    "wiki_crop", url,
    untar=True, cache_dir='.',
    cache_subdir=''
)
dataset_dir = os.path.join(os.path.dirname(dataset), 'wiki_crop')
     

In [3]:
# extract data from wiki.mat
mat = scipy.io.loadmat(os.path.join(dataset_dir,'wiki.mat'))

In [4]:
dob = np.vectorize(lambda x: datetime.datetime.fromordinal(x).year)(
    mat["wiki"]["dob"][0][0][0]
)
photo_taken = mat["wiki"]["photo_taken"][0][0][0]
     

In [5]:
age = (photo_taken-dob).astype(np.float32)

In [6]:
file_path = np.vectorize(lambda x : os.path.join(dataset_dir,x[0]))(
    mat["wiki"]["full_path"][0][0][0]
)

In [7]:
file_age_ds = tf.data.Dataset.from_tensor_slices((file_path,age))
     

def parse_function(filename, label):
    image_string = tf.io.read_file(filename)
    image_decoded = tf.io.decode_jpeg(image_string,channels=1)
    image = tf.image.resize(image_decoded, [256, 256])
    return image, tf.expand_dims(label,0)
     

image_age_ds = file_age_ds.map(parse_function).shuffle(seed=2,buffer_size=64)

In [8]:
dataset_size = image_age_ds.cardinality().numpy()
     

AUTOTUNE = tf.data.AUTOTUNE
train_ds = image_age_ds.take(dataset_size*.6).batch(32).prefetch(AUTOTUNE)
val_ds = image_age_ds.skip(dataset_size*.6).take(dataset_size*.2).batch(32).prefetch(AUTOTUNE)
test_ds = image_age_ds.skip(dataset_size*.8).take(dataset_size*.2).batch(32).prefetch(AUTOTUNE)

In [9]:
model = tf.keras.Sequential([
    tfl.Conv2D(32,(7,7),padding="valid",activation="relu",input_shape=(256,256,1)),
    tfl.MaxPool2D((4,4),strides = 4),
    tfl.Conv2D(64,(3,3),padding = "valid",activation="relu"),
    tfl.MaxPool2D((4,4),strides = 4),
    tfl.Conv2D(128,(3,3),padding = "valid",activation="relu"),
    tfl.MaxPool2D((2,2),strides = 2),
    tfl.Conv2D(256,(1,1),padding= "valid",activation="relu",),
    tfl.MaxPool2D((2,2),strides = 2),
    tfl.Flatten(),
    tfl.Dense(64,activation="relu"),
    tfl.Dense(1)                             
])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 250, 250, 32)      1600      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 62, 62, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 60, 60, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 15, 15, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 13, 13, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 6, 128)        0

In [11]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.MeanAbsoluteError(),
    metrics=['MAE']
)

In [12]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=1,
    batch_size = 1024,
    callbacks=[
        tf.keras.callbacks.TensorBoard(log_dir="logs")
    ]
)

1169/1169 [==============================] - 1640s 1s/step - loss: 14.1831 - MAE: 14.1831 - val_loss: 13.5661 - val_MAE: 13.5661


In [13]:
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

390/390 [==============================] - 177s 364ms/step - loss: 14.0276 - MAE: 14.0276
Loss:  14.027582168579102
Accuracy:  14.027582168579102


In [14]:
x = model.predict(test_ds)

In [23]:
x[0][0]

25.228767

In [24]:
model.save('age.h5')